In [27]:
#!pip install stable_baselines3c
#!pip install shimmy>=2.0

# 1. Import Dependencies

In [28]:
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Types of Spaces

In [29]:
Discrete(3)

Discrete(3)

In [30]:
Box(0,1,shape=(3,3)).sample()

array([[0.665199  , 0.19151694, 0.5780439 ],
       [0.5839929 , 0.32958102, 0.36861345],
       [0.6886991 , 0.75057906, 0.2692577 ]], dtype=float32)

In [31]:
Box(0,255,shape=(3,3), dtype=int).sample()

array([[ 71, 166, 241],
       [239, 252, 104],
       [251, 177, 216]])

In [32]:
Tuple((Discrete(2), Box(0,100, shape=(1,)))).sample()

(1, array([97.08194], dtype=float32))

In [33]:
Dict({'height':Discrete(2), "speed":Box(0,100, shape=(1,))}).sample()

OrderedDict([('height', 1), ('speed', array([31.730486], dtype=float32))])

In [34]:
MultiBinary(4).sample()

array([1, 1, 1, 1], dtype=int8)

In [35]:
MultiDiscrete([5,2,2]).sample()

array([2, 1, 0])

# 3. Building an Environment

In [36]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60

    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0
        # 2 -1 = 1 temperature
        self.state += action -1
        # Reduce shower length by 1 second
        self.shower_length -= 1

        # Calculate reward
        if self.state >=37 and self.state <=39:
            reward =1
        else:
            reward = -1

        # Check if shower is done
        if self.shower_length <= 0:
            done = True
        else:
            done = False

        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}

        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass

    def reset(self):
        # Reset shower temperature
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        # Reset shower time
        self.shower_length = 60
        return self.state

In [37]:
env=ShowerEnv()

In [38]:
env.observation_space.sample()

array([32.65656], dtype=float32)

In [39]:
env.reset()

array([39.])

In [40]:
# from stable_baselines3.common.env_checker import check_env

In [41]:
# check_env(env, warn=True)

# 4. Test Environment

In [42]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-48
Episode:2 Score:-56
Episode:3 Score:-12
Episode:4 Score:-60
Episode:5 Score:-24


In [43]:
env.close()

# 5. Train Model

In [44]:
log_path = os.path.join('Training', 'Logs')

In [45]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [ ]:
model.learn(total_timesteps=400000)

Logging to Training/Logs/PPO_1


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -34.4    |
| time/              |          |
|    fps             | 1241     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | -35.4        |
| time/                   |              |
|    fps                  | 851          |
|    iterations           | 2            |
|    time_elapsed         | 4            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0059240186 |
|    clip_fraction        | 0.0364       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.09        |
|    explained_variance   | -0.000144    |
|    learning_r

# 6. Save Model

In [ ]:
model.save('PPO')

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)